# Convolutional Neural Network


#### Dense Network looks at entire image (global scale)
- it looks at patterns in the entire image - image must be centered, etc.
- it cannot recognize local patterns if they were moved to another part of the image

#### Convolutional Neural Network looks at parts of image (local scale)
- can learn local patterns and find them anywhere in image
- CNN scans image to find features and passes those features to a dense classifier

#### CNN Architecture:
- Not densly connected
- multiple layers used to pick up on complex patterns
    - first layer may pick on edges and lines
    - second layer takes this as input and may start forming shapes
    - last layer might look at shapes and determine if they form a pattern

#### Features Maps:
- A 3D tensor with two spacial axes (width and height) and one depth axis
- CNN layers take feature maps as input and return a new feature map that
represent the presence of specific filters from the previous feature map
- this is called a response map

#### Layer Parameters - CNN defined by two key parameters
- **Filter**: *m* x *n* pattern of pixels that we are looking for in image
    - number of filters in CNN represents how many patterns each layer is
    looking for and what the depth of our response map will be
    - each layer of depth in the reponse map is a matrix containing values
    inicating if each filter was present at that location or not (find by calculating dot product of sample and filter)
    - trainable parameter
- **Sample Size**: each layer is going to examine *n* x *m* blocks of pixels in each image
    - typically, 3x3, or 5x5 blocks (sample size)
    - sampling size is same size as filter
    - layers work by sliding filers of *n* x *m* pixels over every possible position in our image
    and populating a new response map indicating whether or not the filter is present at each location

#### Pooling
- Simplify process by reducing size of feature maps
- takes average, max, or min value in a 2x2 area of feature map, and make that whole area into one pixel in new map

## Image Data
Three Dimensions:
- Image Height
- Image Width
- Color Channels

Color Channels:
- Image is made of several layers, one for the values of each color
- for rgb, red, green, and blue each have their own layers, with pixel values from 0-255

### Dataset

Problem: Classify 10 different everyday objects using the CIFAR Image dataset in tensorflow

It contains 60,000 32x32 color images with 6000 images of each class

It has the following labels:
- Airplane
- Automobile
- Bird
- Cat
- Deer
- Dog
- Frog
- Horse
- Ship
- Truck

In [3]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

# Class names
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

# Load and split dataset
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images = train_images / 255.0
test_images = test_images / 255.0

## Build Model
### CNN Architecture
Common architecture is a stack of Conv2D and MaxPooling2D layers followed by a few dense layers
- stack of convolutional and maxPooling layers extract the features from the image
    - maxPooling layer after each convolutional layer reducing map size with max pixel value
- features are flattened and fed to densly connected layers that determine the class of an image based on features

In [5]:
model = models.Sequential()

# ---------------- Convolutional Base ----------------

# Layer 1: input shape of data is 32x32x3 - will process 32 filters of size 3x3 over input data - will use relu activation function
#          output map of this layer will be 30x30x32 - 30x30 instead of 32x32 bc no padding - last dimen bc 32 filters
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32,32,3)))


# Layer 2: Preform Max Pooling operation using 2x2 samples and a stride of 2 (shrink feature map by factor of 2 )
#          output map will be 15x15x32  - reduce each layer of depth by factor of 2
model.add(layers.MaxPooling2D((2,2)))

# Layer 3: Same as Layer 1, but the input feature map is the output of layer 1 (after max poolng)
#          Also increases frequency of filters from 32 to 64 (can afford this since the feature map size is shrinking from pooling)
#          Output map will be 13x13x64 - lose two pixels bc no padding - 64 filters
model.add(layers.Conv2D(64, (3,3), activation='relu'))

# Layer 4: Same as layer 2
#          Output shape will be 6x6x64 - reduce by factor of 2
model.add(layers.MaxPooling2D((2,2)))

# Layer 5: Same as layer 3
#          Output shape will be 4x4x64  - same as 1 and 3
model.add(layers.Conv2D(64, (3,3), activation='relu'))

# ------------------- Dense Layers -------------------

# Layer 6: Flatten the matrices of feature maps to one dimension - Output shape is 1x1024
model.add(layers.Flatten()) 

# Layer 7: 64 neuron dense layer to predict based on identified features - output later is 1x64 (one output for each neuron)
model.add(layers.Dense(64))

# Layer 8: 10 neuron output layer for 10 classes - output shape is 1x10 (probability distribution of each class)
model.add(layers.Dense(10))

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                                                                 
 conv2d_3 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 6, 6, 64)         0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 4, 4, 64)          36928     
                                                                 
 flatten (Flatten)           (None, 1024)             

### Compile and Train Model
Define the loss function, the optimizer, the metrics to track, and the number of epochs

In [9]:
# Compile model
model.compile(
    optimizer='adam',       # Choose the adam algorithm to preform gradient descent
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),  # Function to claculate the loss
    metrics=['accuracy']    # Keep track of accuracy during training
)

# Train Model
history = model.fit(
    train_images,           # Train Images
    train_labels,           # Train labels
    epochs=8,              # Choose 10 epochs
    validation_data=(test_images, test_labels)  # Testing data
)

Epoch 1/10


2022-12-12 20:30:35.445565: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 614400000 exceeds 10% of free system memory.


1561/1563 [============================>.] - ETA: 0s - loss: 1.4325 - accuracy: 0.4816

2022-12-12 20:30:52.393170: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 122880000 exceeds 10% of free system memory.


1563/1563 [==============================] - 18s 11ms/step - loss: 1.4324 - accuracy: 0.4817 - val_loss: 1.1635 - val_accuracy: 0.5911
Epoch 2/10
1563/1563 [==============================] - 19s 12ms/step - loss: 1.0720 - accuracy: 0.6240 - val_loss: 1.0252 - val_accuracy: 0.6422
Epoch 3/10
1563/1563 [==============================] - 18s 12ms/step - loss: 0.9408 - accuracy: 0.6716 - val_loss: 0.9502 - val_accuracy: 0.6705
Epoch 4/10
1563/1563 [==============================] - 18s 12ms/step - loss: 0.8506 - accuracy: 0.7032 - val_loss: 0.9336 - val_accuracy: 0.6800
Epoch 5/10
1563/1563 [==============================] - 18s 12ms/step - loss: 0.7873 - accuracy: 0.7258 - val_loss: 0.9089 - val_accuracy: 0.6827
Epoch 6/10
1563/1563 [==============================] - 20s 13ms/step - loss: 0.7361 - accuracy: 0.7418 - val_loss: 0.9518 - val_accuracy: 0.6769
Epoch 7/10
1563/1563 [==============================] - 17s 11ms/step - loss: 0.6865 - accuracy: 0.7613 - val_loss: 0.8793 - val_accura